<a href="https://colab.research.google.com/github/gabbiurlaro/aml22-ego/blob/vae/augment_datset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# INSTALL AND GIT


In [1]:
!rm -rf sample_data

!git clone https://github.com/gabbiurlaro/aml22-ego.git aml22-ego
!cd aml22-ego && git checkout vae

Cloning into 'aml22-ego'...
remote: Enumerating objects: 3606, done.
remote: Counting objects: 100% (258/258), done.
remote: Compressing objects: 100% (129/129), done.
remote: Total 3606 (delta 167), reused 206 (delta 128), pack-reused 3348
Receiving objects: 100% (3606/3606), 1.51 GiB | 31.44 MiB/s, done.
Resolving deltas: 100% (2633/2633), done.
Updating files: 100% (45/45), done.
Updating files: 100% (161/161), done.
Branch 'vae' set up to track remote branch 'vae' from 'origin'.
Switched to a new branch 'vae'


In [2]:
# Mount google drive 
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Install conda

!pip install -q condacolab
import condacolab
condacolab.install()

⏬ Downloading https://github.com/conda-forge/miniforge/releases/download/23.1.0-1/Mambaforge-23.1.0-1-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:12
🔁 Restarting kernel...


In [4]:
!mkdir -p /usr/local/envs/egovision
!tar xf /content/drive/MyDrive/egovision.tar.gz --directory=/usr/local/envs/egovision

# Augmentation

In [ ]:
import pickle 
import numpy as np
import pandas as pd
import pywt
import torch
import os
import torch.nn.functional as F
from scipy.interpolate import CubicSpline
import random
import torchvision.transforms
from torch.utils.data import Dataset

train =  pd.DataFrame(pd.read_pickle('/content/aml22-ego/saved_features/ACTIONNET_EMG/EMG_no-clip_ActionNet_train.pkl'))
test =  pd.DataFrame(pd.read_pickle('/content/aml22-ego/saved_features/ACTIONNET_EMG/EMG_no-clip_ActionNet_test.pkl'))
original_train = pd.DataFrame(pd.read_pickle('/content/drive/MyDrive/train_val_EMG/ActionNet_train.pkl'))
original_test = pd.DataFrame(pd.read_pickle('/content/drive/MyDrive/train_val_EMG/ActionNet_test.pkl'))
original_train

In [ ]:
from utils.loaders import ActionNetDataset
sigma = 0.1
wavelet_name = 'db7' #Wavelet name (e.g., Daubechies 4)
decomposition_level = 5 # # Number of decomposition levels
detail_factor = 0 # Scaling factor for modifying detail coefficients
  
num_clips = 5
batch_Size = 1


train =  ActionNetDataset('ActionNet', ['EMG'], 'train', {'stride': 2, 'annotations_path':'/content/drive/MyDrive/train_val_EMG'}, {'EMG': 32}, 5, {'EMG': False},
                                                                       None, load_feat=False, additional_info=True)
#pd.DataFrame(pd.read_pickle('/content/aml22-ego/saved_features/ACTIONNET_EMG/EMG_no-clip_ActionNet_train.pkl'))
test =  ActionNetDataset('ActionNet', ['EMG'], 'test', {'stride': 2, 'annotations_path':'/content/drive/MyDrive/train_val_EMG'}, {'EMG': 32}, 5, {'EMG': False},
                                                                       None, load_feat=False,  additional_info=True)

train.list_file

In [149]:
def wavelet_decomposition(signal, wavelet_name, decomposition_level, detail_factor):
    #print('WD :', signal.shape)
    coeffs = pywt.wavedec(signal, wavelet=wavelet_name, level=decomposition_level)
    cA, cD = coeffs[0], coeffs[1:]  # Approximation and detail coefficients
    
    # Modify detail coefficients
    cD_modified = [detail_factor * cd for cd in cD]
    
    # Reconstruct the augmented signal
    augmented_coeffs = [cA] + cD_modified
    augmented_signal = torch.tensor(pywt.waverec(augmented_coeffs, wavelet=wavelet_name))
    
    return augmented_signal


class WaveletDecompositionTransform:
    def __init__(self, wavelet_name, decomposition_level, detail_factor, num_clips):
        self.wavelet_name = wavelet_name
        self.decomposition_level = decomposition_level
        self.detail_factor = detail_factor
        self.num_clips = num_clips
    
    def __call__(self, sample):
        signals = sample['features_EMG']
        augmented_signals = []
        for i in range(self.num_clips):
          
          augmented_signals.append(wavelet_decomposition(signals[i], self.wavelet_name, self.decomposition_level, self.detail_factor))
        # Create a new augmented sample
        augmented_sample = {
            'features_EMG': torch.stack(augmented_signals).reshape(5, 16, 32, 32),
            'label': sample['label'],
            'uid': sample['uid'],
            'untrimmed_video_name': sample['untrimmed_video_name']
        }
        #print('WD',augmented_sample['features_EMG'].shape)
        
        return augmented_sample

In [150]:
def magnitude_warping(signal, variance=0.01):
    T = signal.size(1)
    #print('MW :', T)
    t = torch.linspace(0, 1, T)  # Equidistant time points
    r = torch.randn(T)  # Random numbers from a normal distribution
    r = torch.clamp(r, -2.0, 2.0)  # Limit the range of random numbers to avoid extreme warping
    
    # Generate a smooth curve using cubic splines
    spline = CubicSpline(t, r)
    cubic_spline = torch.from_numpy(spline(t)).float()
    
    # Elementwise product of the interpolated curve with the signal
    warped_signal = torch.Tensor(signal * (1.0 + variance * cubic_spline))

    return warped_signal

class MagnitudeWarpingTransform:
    def __init__(self, variance, num_clips):
        self.variance= variance
        self.num_clips = num_clips
    
    def __call__(self, sample):
        signals = sample['features_EMG']
        augmented_signals = []
        for i in range(self.num_clips):
         # print('MW1: ', signals.shape)
          augmented_signals.append(magnitude_warping(signals[i], variance=self.variance))
        # Create a new augmented sample
        augmented_sample = {
            'features_EMG': torch.stack(augmented_signals),
            'label': sample['label'],
            'uid': sample['uid'],
            'untrimmed_video_name': sample['untrimmed_video_name']
        }
        
        return augmented_sample

# Example usage
#signal = torch.randn(1024)  # Assuming input signal of size 1024
#warped_signal = magnitude_warping(signal, variance=0.01)

In [153]:
class Fit_Dims:
    def __init__(self, num_clips=5):
        
        self.num_clips = num_clips

    def __call__(self, x):
        signals = x[0]['EMG']
        signals = signals.reshape(5,16, 32, 32)
        #print(signals.shape)
        augmented_sample = {
              'features_EMG':signals,
              'label': x[1],
              'uid': x[3],
              'untrimmed_video_name': x[2]
          }
        #print(augmented_sample['features_EMG'].shape)
        return augmented_sample

In [110]:
signal = train[0][0]['EMG']
signal = signal[0]
print('fd', type(signal), signal.shape)
x = signal.reshape(5,-1,32)
print('fd', type(x), x.shape)

fd <class 'torch.Tensor'> torch.Size([160, 32])
fd <class 'torch.Tensor'> torch.Size([5, 32, 32])


In [166]:
%ls
for i in range(4):
  filename = './aug_original/ActionNet_augmented_clips_' + ts[i]
  with open(f"{filename}_train.pkl", "wb") as file:
            pickle.dump(outs[0][i], file)
  with open(f"{filename}_test.pkl", "wb") as file:
             pickle.dump(outs[1][i], file)

ACTIONNET/  ACTIONNET_EMG/  aug/  aug_original/  EPIC/  reconstructed/


In [173]:
from google.colab import files
!zip -r /content/feats_augs.zip /content/aml22-ego/saved_features/ACTIONNET_EMG_AUG
files.download('/content/feats_augs.zip')

  adding: content/aml22-ego/saved_features/ACTIONNET_EMG_AUG/ (stored 0%)
  adding: content/aml22-ego/saved_features/ACTIONNET_EMG_AUG/job_feature_extraction_test_train.pkl (deflated 11%)
  adding: content/aml22-ego/saved_features/ACTIONNET_EMG_AUG/job_feature_extraction_train_train.pkl (deflated 11%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Plot features

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import random
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
num_clips = 5
# EPIC-KITCHEN and ActionNet

labels = {'EK':{
        0 : "take (get)",
        1 : "put-down (put/place)",
        2 : "open",
        3 : "close",
        4 : "wash (clean)",
        5 : "cut",
        6 : "stir (mix)",
        7 : "pour"
}, 'AN': {
        0 : "Spread",
        1 : "Get/Put",
        2 : "Clear",
        3 : "Slice",
        4 : "Clean",
        5 : "Pour",
        6 : "Load",
        7 : "Peel",
        8 : "Open/Close",
        9 : "Set",
        10 : "Stack",
        11 : "Unload"

}}

colors = {'EK': {
        0 : "#A52A2A",
        1 : "#DAA520",
        2 : "#FF7F50",
        3 : "#7BC8F6",
        4 : "#FFFF14",
        5 : "#76FF7B",
        6 : "#13EAC9",
        7 : "olive"
}, 'AN': {
        0 : "#A52A2A",
        1 : "#DAA520",
        2 : "#FF7F50",
        3 : "#7BC8F6",
        4 : "#FFFF14",
        5 : "#76FF7B",
        6 : "#13EAC9",
        7 : "olive",
        8 : "palegreen",
        9 : "lightpink",
        10 : "darkmagenta",
        11 : "cadetblue"
}}

In [ ]:
data_original = pd.DataFrame(pd.read_pickle('/content/aml22-ego/saved_features/ACTIONNET_EMG/EMG_no-clip_ActionNet_train.pkl')["features"])
data = pd.DataFrame(pd.read_pickle("/content/aml22-ego/saved_features/reconstructed/AUG_VAE_0.001_2023-05-24 16:15:44.696068_train.pkl")["features"])


In [ ]:
data.iloc[0]
ts = ['WD-MW', 'MW', 'WD', 'MW-WD']
data_augmented_train = []
for i in range(4):
  data_augmented_train.append(pd.read_pickle('/content/aml22-ego/saved_features/aug/ActionNet_augmented_clips_'+ ts[i]+'_train.pkl'))
data = pd.DataFrame(data_augmented_train[3]['features'] +data_augmented_train[2]['features']+
                   data_augmented_train[1]['features'] +
                    data_augmented_train[0]['features'] + data_original['features'])
len(data)

In [ ]:
# plot emg features
print(len(data.iloc[0]['features_EMG']))
features = np.array([data.iloc[i].features_EMG[num_clips//2] for i in range(len(data))])
reduced = TSNE().fit_transform(features)
data['x'] = reduced[:, 0]
data['y'] = reduced[:, 1]
for i in range(12): # ek has 8 classes
    filtered = data[data["label"] == i]
    # compute the central frame
    plt.scatter(filtered['x'], filtered['y'], c=colors['AN'][i], label=labels['AN'][i])
plt.legend()
plt.title('EMG train features')
plt.show()

# Train VAE

In [ ]:
%%bash
#VAE EMG train and save

cd aml22-ego && git pull origin vae

PYTHON_PATH=/usr/local/envs/egovision/bin/python

$PYTHON_PATH train_VAE_features_EMG.py action="train_and_save" \
  name="VAE_EMG_2 full-aug lr1e-3 wkld1 sum" \
  config=configs/VAE_save_feat_EMG.yaml \
  dataset.shift=ActionNet-ActionNet \
  wandb_name='vae' \
  wandb_dir='Experiment_logs'  \
  dataset.RGB.data_path=../ek_data/frames \
  dataset.EMG.features_name='ACTIONNET_EMG/EMG_no-clip' \
  models.EMG.model='VAE' \
  models.EMG.lr=1e-3

# Train Classifier

In [186]:
%%bash
ls
cd aml22-ego/
git stash
git pull origin vae

aml22-ego
condacolab_install.log
drive
feats_augs.zip
feats_def.zip
No local changes to save
Updating 85e3809..1d1dfd3
Fast-forward
 configs/classifier_emg.yaml                        |   1 +
 saved_features/.DS_Store                           | Bin 6148 -> 6148 bytes
 .../job_feature_extraction_test_train.pkl          | Bin 0 -> 249144 bytes
 .../job_feature_extraction_train_train.pkl         | Bin 0 -> 2227571 bytes
 train_classifier_EMG.py                            |  66 ++++++++++++++++-----
 utils/loaders.py                                   |   4 +-
 6 files changed, 53 insertions(+), 18 deletions(-)
 create mode 100644 saved_features/ACTIONNET_EMG_AUG/job_feature_extraction_test_train.pkl
 create mode 100644 saved_features/ACTIONNET_EMG_AUG/job_feature_extraction_train_train.pkl


From https://github.com/gabbiurlaro/aml22-ego
 * branch            vae        -> FETCH_HEAD
   8a8750c..1d1dfd3  vae        -> origin/vae


In [2]:
%%bash
#classifier EMG train, validate and save

PYTHON_PATH=/usr/local/envs/egovision/bin/python

$PYTHON_PATH train_classifier_EMG.py action="job_feature_extraction" name="job_feature_extraction" \
  config='configs/classifier_emg.yaml' \
  dataset.shift='ActionNet-ActionNet' \
  train.num_iter=600\
  wandb_name='vae'\
  wandb_dir='Experiment_logs'\
  dataset.RGB.data_path='../ek_data/frames'  \
  models.EMG.model='EMG_classifier' \
  resume_from='./saved_models/EMG_classifier' \
  dataset.EMG.features_name='ACTIONNET_EMG/EMG_Normalized_no-clip' 

bash: line 5: /usr/local/envs/egovision/bin/python: No such file or directory


CalledProcessError: Command 'b'#classifier EMG train, validate and save\n\nPYTHON_PATH=/usr/local/envs/egovision/bin/python\n\n$PYTHON_PATH train_classifier_EMG.py action="job_feature_extraction" name="job_feature_extraction" \\\n  config=\'configs/classifier_emg.yaml\' \\\n  dataset.shift=\'ActionNet-ActionNet\' \\\n  train.num_iter=600\\\n  wandb_name=\'vae\'\\\n  wandb_dir=\'Experiment_logs\'\\\n  dataset.RGB.data_path=\'../ek_data/frames\'  \\\n  models.EMG.model=\'EMG_classifier\' \\\n  resume_from=\'./saved_models/EMG_classifier\' \\\n  dataset.EMG.features_name=\'ACTIONNET_EMG/EMG_Normalized_no-clip\' \n'' returned non-zero exit status 127.